# Agent Short-Term Memory

Short-term memory stores conversation history within a session using checkpointers.

**What you'll learn:**
- Checkpointers persist conversation history
- SQLite for development, PostgreSQL for production
- Thread IDs manage separate sessions
- Access agent state in tools with ToolRuntime
- Modify agent state from tools for context offloading
- Save/load conversation summaries for long conversations

## Checkpointer Comparison

| Type | Use Case | Setup |
|------|----------|-------|
| **SQLite** | Development, testing | Simple file-based |
| **PostgreSQL** | Production, multi-user | Database connection |

In [1]:
import sys
sys.path.append('../')

import os
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent
from langchain.messages import HumanMessage, ToolMessage, SystemMessage
from scripts import base_tools

from langchain.tools import tool, ToolRuntime
from langgraph.types import Command

from pathlib import Path

In [3]:
model = ChatGoogleGenerativeAI(model='gemini-2.5-flash')

system_prompt = """You are a helpful assistant with memory.
- Remember previous messages in the conversation
- Use conversation history when answering questions
- Be concise and accurate"""

## Problem: No Memory

In [4]:
agent = create_agent(model=model, system_prompt=system_prompt)

agent.invoke({'messages': [HumanMessage("My name is Laxmi Kant")]})
response = agent.invoke({'messages': [HumanMessage("What's my name?")]})

response

{'messages': [HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='831a3211-87ad-46a9-b083-422977bb9942'),
  AIMessage(content="I don't know your name. You haven't told it to me yet!", additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bc050-f620-7903-a780-aa72966304ac-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 37, 'output_tokens': 87, 'total_tokens': 124, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 69}})]}

## Short-Term Memory: SQLite

In [8]:
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3

os.makedirs('db', exist_ok=True)

# checkpointer = InMemorySaver()

# get the connection of SQLite db
conn = sqlite3.connect("db/31_checkpoints.db", check_same_thread=False)
checkpointer = SqliteSaver(conn)
checkpointer.setup()

config = {"configurable": {"thread_id": "user_123"}}

agent = create_agent(model=model, 
                     system_prompt=system_prompt,
                     checkpointer=checkpointer)

agent.invoke({'messages': [HumanMessage("My name is Laxmi Kant")]}, config=config)

response = agent.invoke({'messages': [HumanMessage("What's my name?")]}, config=config)

response


{'messages': [HumanMessage(content='My name is Laxmi Kant', additional_kwargs={}, response_metadata={}, id='ef4515c4-d669-45cf-8c48-607ffafac5ed'),
  AIMessage(content="Hello Laxmi Kant! It's nice to meet you. I'll remember your name.", additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bc050-647a-7e02-b797-673e2bfda0eb-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 37, 'output_tokens': 38, 'total_tokens': 75, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 18}}),
  HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='f0a99765-295d-4b5e-99bf-a1766950d194'),
  AIMessage(content='Your name is Laxmi Kant.', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b

In [9]:
agent.get_state(config=config)

StateSnapshot(values={'messages': [HumanMessage(content='My name is Laxmi Kant', additional_kwargs={}, response_metadata={}, id='ef4515c4-d669-45cf-8c48-607ffafac5ed'), AIMessage(content="Hello Laxmi Kant! It's nice to meet you. I'll remember your name.", additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bc050-647a-7e02-b797-673e2bfda0eb-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 37, 'output_tokens': 38, 'total_tokens': 75, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 18}}), HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='f0a99765-295d-4b5e-99bf-a1766950d194'), AIMessage(content='Your name is Laxmi Kant.', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, i

## Short-Term Memory: PostgreSQL

In [7]:
from langgraph.checkpoint.postgres import PostgresSaver
import psycopg
# os.getenv("POSTGRESQL_URL")

In [5]:
# get the connection of PostgreSQL db

pg_conn = psycopg.connect(os.getenv("POSTGRESQL_URL"), autocommit=True)
checkpointer = PostgresSaver(pg_conn)
checkpointer.setup()

config = {"configurable": {"thread_id": "user_123"}}

agent = create_agent(model=model, 
                     system_prompt=system_prompt,
                     checkpointer=checkpointer)

agent.invoke({'messages': [HumanMessage("My name is Laxmi Kant")]}, config=config)

response = agent.invoke({'messages': [HumanMessage("What's my name?")]}, config=config)

response

{'messages': [HumanMessage(content='My name is Laxmi Kant', additional_kwargs={}, response_metadata={}, id='58ec68e6-ee52-455d-a9ff-984ef41563e5'),
  AIMessage(content="Hello Laxmi Kant! I'll remember that.", additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bc072-1a0f-7c52-8fa9-2b1adee9c10a-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 37, 'output_tokens': 61, 'total_tokens': 98, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 50}}),
  HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='43f6e337-b0e8-4302-8450-bfa868b5e8d6'),
  AIMessage(content='Your name is Laxmi Kant.', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bc072-227a-7771-9bf6-4bcf6f68

In [8]:
config = {"configurable": {"thread_id": "user_345"}}

agent = create_agent(model=model, 
                     system_prompt=system_prompt,
                     checkpointer=checkpointer)

agent.invoke({'messages': [HumanMessage("My name is KGP Talkie")]}, config=config)

response = agent.invoke({'messages': [HumanMessage("What's my name?")]}, config=config)

response

{'messages': [HumanMessage(content='My name is KGP Talkie', additional_kwargs={}, response_metadata={}, id='790a5400-764b-4d69-93e6-3f50ef6ad9d6'),
  AIMessage(content="Okay, KGP Talkie. I'll remember that.", additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bc073-8c21-7080-ba86-6cc66ca5718b-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 38, 'output_tokens': 31, 'total_tokens': 69, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 18}}),
  HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='22484df2-422c-42fb-a441-92e42b4d80bb'),
  AIMessage(content='Your name is KGP Talkie.', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bc073-94df-7a83-83ae-c1c2244c

## Context Offloading: Read State in Tools

Access agent state to save conversation summaries for context management.

In [15]:
from dataclasses import dataclass

from langchain.tools import tool, ToolRuntime
from langgraph.types import Command

from pathlib import Path

In [16]:
# get the connection of PostgreSQL db

def get_agent():
    pg_conn = psycopg.connect(os.getenv("POSTGRESQL_URL"), autocommit=True)
    checkpointer = PostgresSaver(pg_conn)
    checkpointer.setup()

    agent = create_agent(model=model, 
                        system_prompt=system_prompt,
                        checkpointer=checkpointer)
    
    return agent

In [17]:
# context is immutable
@dataclass
class UserContext:
    user_id: str
    thread_id: str

In [12]:
agent = get_agent()
config = {"configurable": {"thread_id": "user_345"}}

response = agent.invoke({'messages': [HumanMessage("Summarize my previous conversations")]}, config=config)

response

# agent.get_state(config=config)

{'messages': [HumanMessage(content='My name is KGP Talkie', additional_kwargs={}, response_metadata={}, id='790a5400-764b-4d69-93e6-3f50ef6ad9d6'),
  AIMessage(content="Okay, KGP Talkie. I'll remember that.", additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bc073-8c21-7080-ba86-6cc66ca5718b-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 38, 'output_tokens': 31, 'total_tokens': 69, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 18}}),
  HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='22484df2-422c-42fb-a441-92e42b4d80bb'),
  AIMessage(content='Your name is KGP Talkie.', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bc073-94df-7a83-83ae-c1c2244c

In [13]:
@tool
def save_converation_summary(summary:str, runtime:ToolRuntime):
    """Save conversation summary to disk for context offloading."""

    user_id = runtime.context.user_id
    thread_id = runtime.context.thread_id

    summary_dir = Path(f"data/{user_id}/{thread_id}")
    summary_dir.mkdir(parents=True, exist_ok=True)

    summary_path = summary_dir / "summary.md"
    summary_path.write_text(summary)

    return f"Summary saved to {summary_path}"

In [ ]:
pg_conn = psycopg.connect(os.getenv("POSTGRESQL_URL"), autocommit=True)
checkpointer = PostgresSaver(pg_conn)
checkpointer.setup()

agent = create_agent(model=model, 
                    system_prompt=system_prompt,
                    checkpointer=checkpointer,
                    tools=[base_tools.web_search, base_tools.get_weather, save_converation_summary],
                    context_schema=UserContext)


config = {'configurable': {'thread_id': 'session_1'}}
user_context = UserContext(user_id='kgptalkie', thread_id='session_1')

response = agent.invoke({
    'messages': [HumanMessage('What is the latest news about Apple Stock and then search for the latest weather in Mumbai')]},
    config=config, context=user_context)

response

In [26]:
response = agent.invoke({
    'messages': [HumanMessage('Summarize my previous chats along with the answers with you in detail. and save it using available summary tools.')]},
    config=config, context=user_context)

response

d:\Courses\Udemy\AI Agent Projects\.venv\Lib\site-packages\pydantic\main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='context', input_value=UserContext(user_id='kgpt..., thread_id='session_1'), input_type=UserContext])
  return self.__pydantic_serializer__.to_python(


{'messages': [HumanMessage(content='What is the latest news about Apple Stock and then search for the latest weather in Mumbai', additional_kwargs={}, response_metadata={}, id='8597ada1-a8ca-4687-ae23-fa73ff0a17c5'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'web_search', 'arguments': '{"query": "latest news about Apple Stock"}'}, '__gemini_function_call_thought_signatures__': {'dfa6eb10-622d-4d7a-ab59-40a141111612': 'CqECAXLI2nyJ7Ej0zC0WuDsemBy4XCKbsniq/NPBhwLYhfXrIq7r7RIB+poFVpJNopUz6ynr3hcRtMXeacWaiOm4gxj+NAUaRmRbGjfnkx6zvISJ1rr91Qwzdvl4DajlA0gCjg/ELtGOGeV13gO907FTiOJe8LN4TMMzz/J1H+C0SZoMTpvhxmeDqeseN7h6GzqbAMAfxHuI4VM98BSs3lCVfIFugz+nJTm4/GcfMQBmZ3/k/HopPgJ9hoaYVymUT46UUB1uVxvhHOWs8ZE9RLUP9eCPccJWhEpvPcNOwmpkNqMU1ck7v3jqHyu/DZlYfkA2NQ2wqtJCH4nEPwDIch4J6WM3B8ddgsQA8MMfdZOi8enpfWGhfP4RdxMWMNfuwew6Fw=='}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bc0b

## Context Offloading: Modify State in Tools

Load previous summaries and inject them into agent state.

In [37]:
from langchain.messages import RemoveMessage
from langgraph.graph.message import REMOVE_ALL_MESSAGES

@tool
def load_conversation_summary(runtime: ToolRuntime):
    """Load previous conversation summary from disk."""

    user_id = runtime.context.user_id
    # thread_id = runtime.context.thread_id
    thread_id = runtime.config['configurable']['thread_id']

    summary_dir = Path(f"data/{user_id}/{thread_id}")
    summary_path = summary_dir / "summary.md"
    
    if not summary_path.exists():
        return Command(
            update={
                'messages': [ToolMessage("No previous conversation summary is found", tool_call_id=runtime.tool_call_id)]
            }
        )
    
    summary_text = summary_path.read_text()

    messages = runtime.state.get('messages', [])
    last_ai_message = messages[-1] if messages else None

    new_messages = [
                    RemoveMessage(id=REMOVE_ALL_MESSAGES),
                    HumanMessage(f"Previous conversation summary: {summary_text}")
    ]

    if last_ai_message:
        new_messages.append(last_ai_message)

    new_messages.append(ToolMessage("Successfully loaded previous summary.", tool_call_id=runtime.tool_call_id))

    return Command(
        update={
            'messages': new_messages
        }
    )

In [38]:
pg_conn = psycopg.connect(os.getenv("POSTGRESQL_URL"), autocommit=True)
checkpointer = PostgresSaver(pg_conn)
checkpointer.setup()


agent = create_agent(model=model, 
                    system_prompt=system_prompt,
                    checkpointer=checkpointer,
                    tools=[base_tools.web_search, base_tools.get_weather, save_converation_summary, load_conversation_summary],
                    context_schema=UserContext)


config = {'configurable': {'thread_id': 'session_1'}}
user_context = UserContext(user_id='kgptalkie', thread_id='session_1')

response = agent.invoke({
    'messages': [HumanMessage('Load my previous conversation summary')]},
    config=config, context=user_context)

response

d:\Courses\Udemy\AI Agent Projects\.venv\Lib\site-packages\pydantic\main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='context', input_value=UserContext(user_id='kgpt..., thread_id='session_1'), input_type=UserContext])
  return self.__pydantic_serializer__.to_python(


{'messages': [HumanMessage(content="Previous conversation summary: The conversation began with the user requesting the latest news on Apple Stock and the current weather in Mumbai. I utilized the web search tool to find news about Apple Inc. (AAPL), providing details such as its current trading price (259.70, down 0.10% after-hours; closed at 259.96, down 0.42%), recent developments like Google's Gemini app challenging Apple, analyst calls, a reported AI collaboration between Apple and Google, and Jim Cramer's positive outlook on Apple stock. I also mentioned Apple's market leadership, strong services revenue, and cash reserves. Concurrently, I used the weather tool to retrieve the latest weather for Mumbai, reporting a temperature of 30.2°C (86.4°F), an overcast sky, 46% humidity, and a 17.6 kph (11 mph) Northwest wind. Following this, the user requested to save a summary of the conversations. After a couple of iterations where I sought clarification on the specific content for the su

In [36]:
from langchain_core.messages import RemoveMessage, ToolMessage, HumanMessage
from langgraph.graph.message import REMOVE_ALL_MESSAGES

@tool
def load_conversation_summary(runtime: ToolRuntime):
    """Load previous conversation summary from disk."""
    user_id = runtime.context.user_id
    thread_id = runtime.config['configurable']['thread_id']
    
    summary_path = Path(f"data/{user_id}/{thread_id}/summary.md")
    
    if not summary_path.exists():
        return Command(update={
            "messages": [
                ToolMessage(
                    "No previous summary found.",
                    tool_call_id=runtime.tool_call_id
                )
            ]
        })
    
    # Read summary
    summary_text = summary_path.read_text()
    
    # Get the last AI message (the one that called this tool)
    messages = runtime.state.get("messages", [])
    last_ai_message = messages[-1] if messages else None
    
    # Clear all and rebuild with summary
    new_messages = [
        RemoveMessage(id=REMOVE_ALL_MESSAGES),
        HumanMessage(f"Previous conversation summary:\n{summary_text}"),
    ]
    
    if last_ai_message:
        new_messages.append(last_ai_message)
    
    new_messages.append(
        ToolMessage(
            "Successfully loaded previous summary.",
            tool_call_id=runtime.tool_call_id
        )
    )
    
    return Command(update={"messages": new_messages})

pg_conn = psycopg.connect(os.getenv("POSTGRESQL_URL"), autocommit=True)
checkpointer = PostgresSaver(pg_conn)
checkpointer.setup()


agent = create_agent(model=model, 
                    system_prompt=system_prompt,
                    checkpointer=checkpointer,
                    tools=[base_tools.web_search, base_tools.get_weather, save_converation_summary, load_conversation_summary],
                    context_schema=UserContext)


config = {'configurable': {'thread_id': 'session_2'}}
user_context = UserContext(user_id='kgptalkie', thread_id='session_2')

# response = agent.invoke({
#     'messages': [HumanMessage('What is the latest news about Apple Stock and then search for the latest weather in Mumbai')]},
#     config=config, context=user_context)

# response = agent.invoke({
#     'messages': [HumanMessage('Summarize my previous chats along with the answers with you in detail. and save it using available summary tools.')]},
#     config=config, context=user_context)

response = agent.invoke({
    'messages': [HumanMessage('Load my previous conversation summary')]},
    config=config, context=user_context)

response

{'messages': [HumanMessage(content="Previous conversation summary:\nThe user initially requested to load a previous conversation summary, but none was found. Subsequently, the user asked for the latest news on Apple Stock and the current weather in Mumbai. I provided a summary of Apple Stock news from CNBC and Yahoo Finance, including topics like xAI probes, Google's Gemini challenging Apple, AI's impact on software, analyst calls, and Apple's sales job cuts. For Mumbai, I reported the current weather as 31.4°C (88.5°F), overcast, with a real feel of 33.1°C (91.5°F), and wind from the NW at 22 km/h (13.6 mph).", additional_kwargs={}, response_metadata={}, id='77b5763f-aa76-466f-81c5-e5d5abbf1aa7'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'load_conversation_summary', 'arguments': '{}'}, '__gemini_function_call_thought_signatures__': {'d42e1bda-7633-4d56-b3f7-77fded30c457': 'CsEBAXLI2nzJdHNwta8ZhS1RLhR/6PDDvlole7uq4lYH6i4/uJEJVqSMPma6U7WM1andzUzpAD2/i+/R8n3WN